## Least Square (for $l=1$ Toy Model)

We successfully downloaded, loaded and filtered the data for 5 parameters solution, we defined the toroidal and spheroidal functions ($T_{lm}$ and $S_{lm}$), respectively modelling the right ascension (ra) and declination (dec). For $\alpha \in [0, 2\pi]$ and $\delta \in [-\pi/2,\pi/2]$ we visualised the VSH vector fields. We now want to perform a MLE on the dataset.

To do so, we will follow closely the procedure presented in the main paper (Gaia Early Data Release 3 Acceleration of the Solar System from Gaia astrometry). This assumes that the noise follows a Gaussian model, i.e. the astrometric measurement errors (in proper motion, parallax, etc.) are:
- Unbiased (zero mean),
- Independent between different sources (quasars),
- With known standard deviation and correlations, as provided in Gaia EDR3.

This allows the least-square estimation framework and the statistical significance tests, in particular using the $\chi^2$ distributions for assessing power in VSH.

Recall Eq. 5 amd 7:
$$
V(\alpha, \delta) = \sum_{l=1}^{l_{\text{max}}} \left( t_{l0} T_{l0} + s_{l0} S_{l0}
+ 2 \sum_{m=1}^{l} \left( t_{lm}^{\mathbb{R}} T_{lm}^{\mathbb{R}} - t_{lm}^{\mathbb{I}} T_{lm}^{\mathbb{I}} + s_{lm}^{\mathbb{R}} S_{lm}^{\mathbb{R}} - s_{lm}^{\mathbb{I}} S_{lm}^{\mathbb{I}} \right) \right)\tag{5}
$$

$$
X^2 = \begin{bmatrix}
\Delta\mu_{\alpha^*} & \Delta\mu_{\delta} \\
\end{bmatrix}
\begin{bmatrix}
\sigma_{\mu_{\alpha^*}}^2 & \rho_{\mu}\sigma_{\mu_{\alpha^*}}\sigma_{\mu_{\delta}} \\
\rho_{\mu}\sigma_{\mu_{\alpha^*}}\sigma_{\mu_{\delta}} & \sigma_{\mu_{\delta}}^2
\end{bmatrix}
\begin{bmatrix}
\Delta\mu_{\alpha^*} \\ \Delta\mu_{\delta} 
\end{bmatrix}\tag{7}
$$

where:
- $\Delta\mu_{\alpha^*} = \mu_{\alpha^* \text{obs}} - V_{\alpha^* \text{model}}$ is the difference between observed and predicted proper motion right ascension (ra).
- $\Delta\mu_{\delta} = \mu_{\delta \text{obs}} - V_{\delta \text{model}}$ is the difference between observed and predicted proper motion declination (dec).

Since each proper motion componet is assumed to follow a Gaussian distribution, MLE simplifies to a weighted least squares. Hence our objective is to minimise Eq. 7:

$$
\sum_k \begin{bmatrix}
\Delta\mu_{\alpha^*} & \Delta\mu_{\delta} \\
\end{bmatrix}
\begin{bmatrix}
\sigma_{\mu_{\alpha^*}}^2 & \rho_{\mu}\sigma_{\mu_{\alpha^*}}\sigma_{\mu_{\delta}} \\
\rho_{\mu}\sigma_{\mu_{\alpha^*}}\sigma_{\mu_{\delta}} & \sigma_{\mu_{\delta}}^2
\end{bmatrix}
\begin{bmatrix}
\Delta\mu_{\alpha^*} \\ \Delta\mu_{\delta} 
\end{bmatrix}
$$

In [3]:
import jax 
import jax.numpy as jnp
import math
from jax import jit, vmap
from functools import partial, lru_cache
from src.models.module2 import*
from jax import random
import pandas as pd
from iminuit import Minuit # to perform least square

### Generate Fake Data
Before we proceed with the real dataset, we want to generate some “fake” data to test our functions and minimiser.

In [2]:
# Load the data

# Choose fixed t_lm and s_lm values (mas/yr)
theta_gen = jnp.array([
    -0.2,   # t_10
     0.05,  # Re(t_11)
     0.01,  # Im(t_11)
     0.3,   # s_10
    -0.04,  # Re(s_11)
     0.02   # Im(s_11)
])

# Generate N random points on the sphere (RA, Dec in radians)
key = random.PRNGKey(0)
N = 5000

ra = random.uniform(key, shape=(N,), minval=0.0, maxval=2 * jnp.pi)
dec = jnp.arcsin(random.uniform(key, shape=(N,), minval=-1.0, maxval=1.0))  # uniform on sphere

angles_gen = jnp.stack([ra, dec])  # shape (2, N)

# Use model to get proper motion vectors, then project to RA/Dec components
mu_alpha = []
mu_delta = []

for i in range(N):
    alpha_i = ra[i]
    delta_i = dec[i]
    e_a, e_d = basis_vectors(alpha_i, delta_i)

    V = toy_model_l_1(alpha_i, delta_i, theta_gen, grid=False)
    mu_alpha.append(jnp.vdot(V, e_a).real)
    mu_delta.append(jnp.vdot(V, e_d).real)

mu_alpha = jnp.array(mu_alpha)  # shape (N,)
mu_delta = jnp.array(mu_delta)

# Add Gaussian noise
noise_level = 0.03  # mas/yr
key1, key2 = random.split(key)
mu_alpha_noisy = mu_alpha + random.normal(key1, shape=(N,)) * noise_level
mu_delta_noisy = mu_delta + random.normal(key2, shape=(N,)) * noise_level

# Pack into obs and error arrays
obs_gen = jnp.stack([mu_alpha_noisy, mu_delta_noisy])  # shape (2, N)
error_gen = jnp.stack([
    jnp.ones(N)*noise_level,       # pmra_error
    jnp.ones(N)*noise_level,       # pmdec_error
    jnp.zeros(N)                    # pmra_pmdec_corr
])

In [3]:
# Bind fixed arguments into a new function
bound_least_square = partial(toy_least_square, angles_gen, obs_gen, error_gen)

# Now Minuit only sees the 6 free parameters
m = Minuit(bound_least_square,
           t_10=0.0, t_11r=0.0, t_11i=0.0,
           s_10=0.0, s_11r=0.0, s_11i=0.0)

m.errordef=Minuit.LEAST_SQUARES
lim = 1.2
m.limits['t_10'] = (-lim,lim)
m.limits['t_11r'] = (-lim,lim)
m.limits['t_11i'] = (-lim,lim)
m.limits['s_10'] = (-lim,lim)
m.limits['s_11r'] = (-lim,lim)
m.limits['s_11i'] = (-lim,lim)

m.migrad()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 1.028e+04                  │             Nfcn = 1258              │
│ EDM = 0.00189 (Goal: 0.0002)     │            time = 4.1 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│         INVALID Minimum          │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           ABOVE call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│           Hesse FAILED           │           ABOVE call limit           │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬───────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name  │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼───────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ t_10  │-1.9775e-1 │ 0.0000e-1 │            │            │  -1.2   │   1.2   │       │
│ 1 │ t_11r │ 5.0282e-2 │ 0.0000e-2 │            │            │  -1.2   │   1.2   │       │
│ 2 │ t_11i │ 1.6526e-2 │ 0.0000e-2 │            │            │  -1.2   │   1.2   │       │
│ 3 │ s_10  │ 2.9948e-1 │ 0.0000e-1 │            │            │  -1.2   │   1.2   │       │
│ 4 │ s_11r │-4.0785e-2 │ 0.0000e-2 │            │            │  -1.2   │   1.2   │       │
│ 5 │ s_11i │ 1.6186e-2 │ 0.0000e-2 │            │            │  -1.2   │   1.2   │       │
└───┴───────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘

Quick comparison to predicted vs true parameters values

In [4]:
theta_fit = jnp.array([m.values[k] for k in m.parameters])
print("Fitted parameters values:")
print(theta_fit)
print("True values:")
print(theta_gen)

Fitted parameters values:
[-0.1977529   0.05028246  0.01652588  0.29948458 -0.04078477  0.01618575]
True values:
[-0.2   0.05  0.01  0.3  -0.04  0.02]


# Toy Model on True Dataset
Now that we have verified that the minimisation of the least square is working correctly on the generated dataset, we can use the true dataset.

In [5]:
# Load the data
df_loaded = pd.read_csv("qso_full_data.csv")
df = df_loaded[df_loaded["astrometric_params_solved"]==31]

In [6]:
# Preparing dataset
ra_rad = jnp.deg2rad(jnp.array(df["ra"].values))
dec_rad = jnp.deg2rad(jnp.array(df["dec"].values))

# Group as [ra, dec] shape = (2, N)
angles = jnp.stack([ra_rad, dec_rad])

# Prepare observed proper motions
pmra = jnp.array(df["pmra"].values)
pmdec = jnp.array(df["pmdec"].values)

# Group as [pmra, pmdec] shape = (2, N)
obs = jnp.stack([pmra, pmdec])

# Prepare error
pmra_error = jnp.array(df["pmra_error"].values)
pmdec_error = jnp.array(df["pmdec_error"].values)
corr = jnp.array(df["pmra_pmdec_corr"].values)

# Group as [pmra_error, pmdec_error, pmra_pmdec_corr] shape = (3, N)
error = jnp.stack([pmra_error, pmdec_error, corr])

In [7]:
# Bind fixed arguments into a new function
bound_least_square = partial(toy_least_square, angles, obs, error)

# Now Minuit only sees the 6 free parameters
m = Minuit(bound_least_square,
           t_10=0.0, t_11r=0.0, t_11i=0.0,
           s_10=0.0, s_11r=0.0, s_11i=0.0)

m.errordef=Minuit.LEAST_SQUARES
lim = 0.05
m.limits['t_10'] = (-lim,lim)
m.limits['t_11r'] = (-lim,lim)
m.limits['t_11i'] = (-lim,lim)
m.limits['s_10'] = (-lim,lim)
m.limits['s_11r'] = (-lim,lim)
m.limits['s_11i'] = (-lim,lim)

m.migrad()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 2.728e+06                  │             Nfcn = 2817              │
│ EDM = 0.888 (Goal: 0.0002)       │           time = 12.9 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│         INVALID Minimum          │   ABOVE EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │     Covariance FORCED pos. def.      │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬───────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name  │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼───────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ t_10  │  -0.003   │   0.009   │            │            │  -0.05  │  0.05   │       │
│ 1 │ t_11r │  -0.010   │   0.011   │            │            │  -0.05  │  0.05   │       │
│ 2 │ t_11i │  -0.006   │   0.008   │            │            │  -0.05  │  0.05   │       │
│ 3 │ s_10  │  -0.005   │   0.004   │            │            │  -0.05  │  0.05   │       │
│ 4 │ s_11r │   0.001   │   0.009   │            │            │  -0.05  │  0.05   │       │
│ 5 │ s_11i │   0.021   │   0.010   │            │            │  -0.05  │  0.05   │       │
└───┴───────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌───────┬───────────────────────────────────────────────────────┐
│       │     t_10    t_11r    t_11i     s_10    s_11r    s_11i │
├───────┼───────────────────────────────────────────────────────┤
│  t_10 │ 7.98e-05  0.10e-3  0.07e-3 0.036e-3  0.08e-3  0.09e-3 │
│ t_11r │  0.10e-3 0.000115  0.09e-3 0.043e-3  0.09e-3  0.10e-3 │
│ t_11i │  0.07e-3  0.09e-3 6.76e-05 0.033e-3  0.07e-3  0.08e-3 │
│  s_10 │ 0.036e-3 0.043e-3 0.033e-3 1.65e-05 0.035e-3 0.039e-3 │
│ s_11r │  0.08e-3  0.09e-3  0.07e-3 0.035e-3 7.72e-05  0.08e-3 │
│ s_11i │  0.09e-3  0.10e-3  0.08e-3 0.039e-3  0.08e-3 9.41e-05 │
└───────┴───────────────────────────────────────────────────────┘

Checking results for $l=1$

In [8]:
theta_fit = jnp.array([m.values[k] for k in m.parameters])
result_spheroidal = spheroidal_vector_summary(theta_fit[3], theta_fit[4], theta_fit[5])
result_torodoidal = toroidal_vector_summary(theta_fit[0], theta_fit[1], theta_fit[2])

In [9]:
result_spheroidal

{'G_vector (mas/yr)': Array([-0.00024633, -0.0101188 , -0.00182672], dtype=float32),
 'Magnitude (μas/yr)': Array(10.285318, dtype=float32),
 'RA (deg)': Array(268.6055, dtype=float32),
 'Dec (deg)': Array(-10.230245, dtype=float32)}

# For Albitrary Choice of $l_{max}$

### Generate Fake Data 

In [10]:
def generate_random_theta(lmax, amplitude=0.01, seed=0):
    key = jax.random.PRNGKey(seed)
    n_params = count_vsh_coeffs(lmax)
    theta = jax.random.uniform(key, shape=(n_params,), minval=-amplitude, maxval=amplitude)
    return theta

In [23]:
lmax = 2
# Choose fixed t_lm and s_lm values (mas/yr)
theta_gen = generate_random_theta(lmax, amplitude=0.06, seed=0)

key = random.PRNGKey(0)

# Generate N random points on the sphere (RA, Dec in radians)
N = 5000

ra = random.uniform(key, shape=(N,), minval=0.0, maxval=2 * jnp.pi)
dec = jnp.arcsin(random.uniform(key, shape=(N,), minval=-1.0, maxval=1.0))  # uniform on sphere

angles_gen = jnp.stack([ra, dec])  # shape (2, N)

# Use model to get proper motion vectors, then project to RA/Dec components
mu_alpha = []
mu_delta = []

for i in range(N):
    alpha_i = ra[i]
    delta_i = dec[i]
    e_a, e_d = basis_vectors(alpha_i, delta_i)

    V = model_vsh(alpha_i, delta_i, theta_gen, lmax, grid=False)
    mu_alpha.append(jnp.vdot(V, e_a).real)
    mu_delta.append(jnp.vdot(V, e_d).real)

mu_alpha = jnp.array(mu_alpha)  # shape (N,)
mu_delta = jnp.array(mu_delta)

# Add Gaussian noise
noise_level = 0.03  # mas/yr
key1, key2 = random.split(key)
mu_alpha_noisy = mu_alpha + random.normal(key1, shape=(N,)) * noise_level
mu_delta_noisy = mu_delta + random.normal(key2, shape=(N,)) * noise_level

# Pack into obs and error arrays
obs = jnp.stack([mu_alpha_noisy, mu_delta_noisy])  # shape (2, N)
error = jnp.stack([
    jnp.ones(N) * noise_level,       # pmra_error
    jnp.ones(N) * noise_level,       # pmdec_error
    jnp.zeros(N)                    # pmra_pmdec_corr
])

In [24]:
theta_gen

Array([ 0.05372004,  0.05742959, -0.02012502, -0.00375978,  0.00838665,
       -0.04013964, -0.02277664,  0.02273766,  0.02961199, -0.03947825,
        0.05824246, -0.05696608,  0.01680502,  0.0075229 ,  0.04790566,
        0.0521445 ], dtype=float32)

In [29]:
lmax = 2
total_params = count_vsh_coeffs(lmax) 

# Flat vector theta: [t10, ..., t_lmaxm, s10, ..., s_lmaxm]
theta_init = jnp.zeros(total_params)

# Fix everything except theta
#bound_least_square = partial(least_square, data, obs, error, lmax=lmax, grid=False)

def least_square_wrapper(*theta_flat):
    theta = jnp.array(theta_flat)  # reconstructs the vector from scalars
    return least_square(angles_gen, obs, error, theta, lmax=lmax, grid=False)


m = Minuit(least_square_wrapper, *theta_init)

m.errordef = Minuit.LIKELIHOOD
for i, name in enumerate(m.parameters):
    m.limits[name] = (-0.07, 0.07)


m.migrad()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 1.027e+04                  │             Nfcn = 4066              │
│ EDM = 0.114 (Goal: 0.0001)       │            time = 7.0 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│         INVALID Minimum          │   ABOVE EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │     Covariance FORCED pos. def.      │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ x0   │  0.0548   │  0.0018   │            │            │  -0.07  │  0.07   │       │
│ 1 │ x1   │  50.5e-3  │  0.9e-3   │            │            │  -0.07  │  0.07   │       │
│ 2 │ x2   │  -0.004   │   0.005   │            │            │  -0.07  │  0.07   │       │
│ 3 │ x3   │  0.0008   │  0.0014   │            │            │  -0.07  │  0.07   │       │
│ 4 │ x4   │   0.027   │   0.012   │            │            │  -0.07  │  0.07   │       │
│ 5 │ x5   │  -0.044   │   0.008   │            │            │  -0.07  │  0.07   │       │
│ 6 │ x6   │  -0.008   │   0.012   │            │            │  -0.07  │  0.07   │       │
│ 7 │ x7   │  0.0033   │  0.0034   │            │            │  -0.07  │  0.07   │       │
│ 8 │ x8   │  0.0363   │  0.0027   │            │            │  -0.07  │  0.07   │       │
│ 9 │ x9   │  -0.0356  │  0.0026   │            │            │  -0.07  │  0.07   │       │
│ 10│ x10  │  0.0663   │  0.0013   │            │            │  -0.07  │  0.07   │       │
│ 11│ x11  │  -0.038   │   0.012   │            │            │  -0.07  │  0.07   │       │
│ 12│ x12  │  0.0070   │  0.0021   │            │            │  -0.07  │  0.07   │       │
│ 13│ x13  │  0.0125   │  0.0024   │            │            │  -0.07  │  0.07   │       │
│ 14│ x14  │   0.045   │   0.011   │            │            │  -0.07  │  0.07   │       │
│ 15│ x15  │  0.0486   │  0.0014   │            │            │  -0.07  │  0.07   │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌─────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│     │        x0        x1        x2        x3        x4        x5        x6        x7        x8        x9       x10       x11       x12       x13       x14       x15 │
├─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│  x0 │  3.39e-06   -0.3e-6    6.7e-6    0.9e-6  -18.4e-6  -11.8e-6  -17.7e-6    4.9e-6   -4.1e-6   -3.2e-6         0  -18.0e-6    2.4e-6   -3.1e-6  -17.2e-6    1.4e-6 │
│  x1 │   -0.3e-6  8.88e-07   -1.0e-6    0.2e-6    2.6e-6    1.7e-6    2.5e-6   -0.7e-6    0.3e-6    0.5e-6   -0.3e-6    2.5e-6   -0.3e-6    0.3e-6    2.4e-6   -0.3e-6 │
│  x2 │    6.7e-6   -1.0e-6  2.19e-05    1.5e-6 -0.053e-3 -0.033e-3 -0.051e-3  0.013e-3    -11e-6     -9e-6   -0.7e-6 -0.052e-3      7e-6     -9e-6 -0.050e-3    3.9e-6 │
│  x3 │    0.9e-6    0.2e-6    1.5e-6  2.09e-06   -4.2e-6   -2.7e-6   -4.0e-6    1.1e-6   -1.0e-6   -0.7e-6   -0.1e-6   -4.1e-6    0.6e-6   -0.8e-6   -3.9e-6   -0.1e-6 │
│  x4 │  -18.4e-6    2.6e-6 -0.053e-3   -4.2e-6  0.000147   0.09e-3   0.14e-3 

In [30]:
theta_fit = jnp.array([m.values[k] for k in m.parameters])
print("Fitted parameters values:")
print(theta_fit)
print("True values:")
print(theta_gen)

Fitted parameters values:
[ 0.05480551  0.05049424 -0.00404864  0.00078248  0.0271537  -0.04445808
 -0.00768095  0.00331257  0.03634993 -0.03561842  0.06628404 -0.03780759
  0.00700449  0.0124701   0.04529925  0.04862593]
True values:
[ 0.05372004  0.05742959 -0.02012502 -0.00375978  0.00838665 -0.04013964
 -0.02277664  0.02273766  0.02961199 -0.03947825  0.05824246 -0.05696608
  0.01680502  0.0075229   0.04790566  0.0521445 ]


# On Real Data for $l=2$

In [31]:
# Preparing dataset
ra_rad = jnp.deg2rad(jnp.array(df["ra"].values))
dec_rad = jnp.deg2rad(jnp.array(df["dec"].values))

# Group as [ra, dec] shape = (2, N)
angles = jnp.stack([ra_rad, dec_rad])

# Prepare observed proper motions
pmra = jnp.array(df["pmra"].values)
pmdec = jnp.array(df["pmdec"].values)

# Group as [pmra, pmdec] shape = (2, N)
obs = jnp.stack([pmra, pmdec])

# Prepare error
pmra_error = jnp.array(df["pmra_error"].values)
pmdec_error = jnp.array(df["pmdec_error"].values)
corr = jnp.array(df["pmra_pmdec_corr"].values)

# Group as [pmra_error, pmdec_error, pmra_pmdec_corr] shape = (3, N)
error = jnp.stack([pmra_error, pmdec_error, corr])

In [89]:
limits = vsh_minuit_limits(lmax=2, t_bound=0.01, s_bound=0.0085)
limits

{'x0': (-0.01, 0.01),
 'x1': (-0.0085, 0.0085),
 'x2': (-0.01, 0.01),
 'x3': (-0.01, 0.01),
 'x4': (-0.0085, 0.0085),
 'x5': (-0.0085, 0.0085),
 'x6': (-0.01, 0.01),
 'x7': (-0.0085, 0.0085),
 'x8': (-0.01, 0.01),
 'x9': (-0.01, 0.01),
 'x10': (-0.0085, 0.0085),
 'x11': (-0.0085, 0.0085),
 'x12': (-0.01, 0.01),
 'x13': (-0.01, 0.01),
 'x14': (-0.0085, 0.0085),
 'x15': (-0.0085, 0.0085)}

In [102]:
lmax = 2
total_params = count_vsh_coeffs(lmax) 
limits = vsh_minuit_limits(lmax=2, t_bound=0.01, s_bound=0.009)

# Flat vector theta: [t10, ..., t_lmaxm, s10, ..., s_lmaxm]
theta_init = jnp.zeros(total_params)

# Fix everything except theta
#bound_least_square = partial(least_square, data, obs, error, lmax=lmax, grid=False)

def least_square_wrapper(*theta_flat):
    theta = jnp.array(theta_flat)  # reconstructs the vector from scalars
    return least_square(angles, obs, error, theta, lmax=lmax, grid=False)


m = Minuit(least_square_wrapper, *theta_init)

m.errordef = Minuit.LEAST_SQUARES
for i, name in enumerate(m.parameters):
    m.limits[name] = limits[name]


m.migrad()
m.params

,Name,Value,Hesse Error,Minos Error-,Minos Error+,Limit-,Limit+,Fixed
0,x0,-0.0016,0.0010,,,-0.01,0.01,
1,x1,-6.3e-3,0.9e-3,,,-0.009,0.009,
2,x2,-10.0e-3,0.4e-3,,,-0.01,0.01,
3,x3,-0.0049,0.0013,,,-0.01,0.01,
4,x4,0.0031,0.0014,,,-0.009,0.009,
5,x5,9.0e-3,0.1e-3,,,-0.009,0.009,
6,x6,6.5e-3,0.8e-3,,,-0.01,0.01,
7,x7,-3.4e-3,0.9e-3,,,-0.009,0.009,
8,x8,0.0016,0.0014,,,-0.01,0.01,
9,x9,-0.0037,0.0014,,,-0.01,0.01,


In [103]:
theta_fit = jnp.array([m.values[k] for k in m.parameters])
print(theta_fit)
print(len(theta_fit))

[-0.00162393 -0.00626221 -0.00995266 -0.00492921  0.00306022  0.0089999
  0.00650263 -0.00344735  0.00164087 -0.00373338  0.00727557 -0.00672269
 -0.00461406 -0.00478772 -0.00487809 -0.00541382]
16


In [104]:
result = spheroidal_vector_summary(theta_fit[1], theta_fit[4], theta_fit[5])

for key, value in result.items():
    print(f"{key:25}: {value}")

G_vector (mas/yr)        : [-0.00149523 -0.00439737 -0.00216356]
Magnitude (μas/yr)       : 5.123826503753662
RA (deg)                 : 251.2205352783203
Dec (deg)                : -24.976987838745117


## On Real Data for l=3

In [94]:
lmax = 3
total_params = count_vsh_coeffs(lmax) 
limits = vsh_minuit_limits(lmax=3, t_bound=0.01, s_bound=0.0085)

# Flat vector theta: [t10, ..., t_lmaxm, s10, ..., s_lmaxm]
theta_init = jnp.zeros(total_params)

# Fix everything except theta
#bound_least_square = partial(least_square, data, obs, error, lmax=lmax, grid=False)

def least_square_wrapper(*theta_flat):
    theta = jnp.array(theta_flat)  # reconstructs the vector from scalars
    return least_square(angles, obs, error, theta, lmax=lmax, grid=False)


m = Minuit(least_square_wrapper, *theta_init)

m.errordef = Minuit.LEAST_SQUARES
for i, name in enumerate(m.parameters):
    m.limits[name] = limits[name]


m.migrad()
m.params

,Name,Value,Hesse Error,Minos Error-,Minos Error+,Limit-,Limit+,Fixed
0,x0,0.2e-3,0.7e-3,,,-0.01,0.01,
1,x1,-0.0079,0.0012,,,-0.0085,0.0085,
2,x2,-0.010,0.016,,,-0.01,0.01,
3,x3,-0.0070,0.0013,,,-0.01,0.01,
4,x4,0.001,0.005,,,-0.0085,0.0085,
5,x5,8.5e-3,0.2e-3,,,-0.0085,0.0085,
6,x6,0.0068,0.0011,,,-0.01,0.01,
7,x7,-0.0045,0.0016,,,-0.0085,0.0085,
8,x8,0.0026,0.0024,,,-0.01,0.01,
9,x9,-0.0016,0.0018,,,-0.01,0.01,


In [95]:
theta_fit = jnp.array([m.values[k] for k in m.parameters])
print(theta_fit)
print(len(theta_fit))

[ 0.00021108 -0.00793433 -0.00998112 -0.00701757  0.00101438  0.00849999
  0.0067874  -0.00448191  0.00257037 -0.00157097  0.00829416 -0.0084946
 -0.00351969 -0.00700597 -0.00849195 -0.0053791   0.00356829  0.00239573
 -0.00898597 -0.00503884  0.00248674 -0.00301224 -0.00591888  0.00811617
  0.00398087  0.00295184  0.00557932  0.00997242  0.00394414  0.00541453]
30


In [96]:
result = spheroidal_vector_summary(theta_fit[1], theta_fit[4], theta_fit[5])

for key, value in result.items():
    print(f"{key:25}: {value}")


G_vector (mas/yr)        : [-0.00049563 -0.00415312 -0.00274127]
Magnitude (μas/yr)       : 5.000856399536133
RA (deg)                 : 263.194580078125
Dec (deg)                : -33.24082946777344


In [97]:
chi2 = m.fval
ndof = 1215942*2 - 30
chi2_red = chi2 / ndof
print(chi2_red) # if chi2_red ~ 1 good fit

1.1216382644681795
